In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold

from sklearn.preprocessing import StandardScaler

from scipy import stats

%matplotlib inline

I:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [186]:
train_data = pd.read_csv('house_price/train.csv')
train_data = train_data.drop(['Id'] , axis = 1)

In [187]:
#++++++++++++++++
#++++++++++++++++
test_data = pd.read_csv('house_price/test.csv')
test_id = test_data.Id
test_data = test_data.drop('Id' , axis = 1)

In [188]:
#对saleprice进行除偏
train_data.SalePrice = train_data.SalePrice.apply(np.log1p)

In [189]:
numerical_columns = train_data.dtypes[train_data.dtypes != object].index #含有预测值 SalePrice
categorical_columns = train_data.dtypes[train_data.dtypes == object].index

In [58]:
#==================
#==================
#==================
#下面的eda全是numerical features

In [190]:
missing_columns_num = (train_data[numerical_columns].isnull().sum()[train_data[numerical_columns].isnull().sum().values>0]).index
#含有缺失值的numerical columns
missing_columns_num

Index(['LotFrontage', 'MasVnrArea', 'GarageYrBlt'], dtype='object')

In [191]:
train_data.LotFrontage = train_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
train_data.MasVnrArea = train_data.MasVnrArea.fillna(0)
train_data.GarageYrBlt = train_data.GarageYrBlt.fillna(0)

In [192]:
#++++++++++++++++++++
#++++++++++++++++++++
test_categorical_columns = categorical_columns.copy()

test_numerical_columns = list(numerical_columns)
test_numerical_columns.remove('SalePrice')
test_missing_columns_num = (test_data[test_numerical_columns].isnull().sum()[test_data[test_numerical_columns].isnull().sum().values>0]).index
test_missing_columns_num

Index(['LotFrontage', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
       'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageYrBlt',
       'GarageCars', 'GarageArea'],
      dtype='object')

In [193]:
#++++++++++++++++++++
#++++++++++++++++++++
test_data.LotFrontage = test_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
test_data.MasVnrArea =  test_data.MasVnrArea.fillna(0)
test_data.GarageYrBlt = test_data.GarageYrBlt.fillna(0)

test_data.BsmtFinSF1 = test_data.BsmtFinSF1.fillna(0)
test_data.BsmtFinSF2 = test_data.BsmtFinSF2.fillna(0)
test_data.BsmtUnfSF = test_data.BsmtUnfSF.fillna(0)
test_data.TotalBsmtSF = test_data.TotalBsmtSF.fillna(0)
test_data.BsmtFullBath = test_data.BsmtFullBath.fillna(0)
test_data.BsmtHalfBath = test_data.BsmtHalfBath.fillna(0)
test_data.GarageCars = test_data.GarageCars.fillna(0)
test_data.GarageArea = test_data.GarageArea.fillna(0)


In [ ]:
#==========
#==========
#==========
#下面是处理numerical的离群值

In [194]:
train_data.loc[train_data.TotalBsmtSF>6000].index #根据上图找离群值
train_data = train_data.drop(1298) #删除离群值

In [195]:
train_data.loc[train_data.GrLivArea>4500,'GrLivArea'].index #找离群点
train_data = train_data.drop([523]) #上面已经把1298删除了
#注意删除前面的离群值后 后面的样本序号会变

In [150]:
no_missing_columns_num = list(set(numerical_columns) - set(missing_columns_num)) #没有缺失值的numerical features

In [196]:
#==================
#==================
#==================
#==================
#下面处理标称型特征
categorical_columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [197]:
#有缺失值的categorical features
missing_columns_cat = categorical_columns[train_data[categorical_columns].isnull().sum().values != 0]
missing_columns_cat

Index(['Alley', 'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'Electrical', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature'],
      dtype='object')

In [198]:
#+++++++++++++++++++
#+++++++++++++++++++
test_missing_columns_cat = test_categorical_columns[test_data[test_categorical_columns].isnull().sum().values != 0]
test_missing_columns_cat

Index(['MSZoning', 'Alley', 'Utilities', 'Exterior1st', 'Exterior2nd',
       'MasVnrType', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'KitchenQual', 'Functional', 'FireplaceQu',
       'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC',
       'Fence', 'MiscFeature', 'SaleType'],
      dtype='object')

In [199]:
#++++++++++++++++++++++
#++++++++++++++++++++++
test_data.MSZoning = test_data.MSZoning.fillna(test_data.MSSubClass.value_counts().index[0])
test_data.Exterior1st = test_data.Exterior1st.fillna(test_data.Exterior1st.value_counts().index[0])
test_data.Exterior2nd = test_data.Exterior2nd.fillna(test_data.Exterior2nd.value_counts().index[0])
test_data.KitchenQual = test_data.KitchenQual.fillna(test_data.KitchenQual.value_counts().index[0])
test_data.Functional = test_data.Functional.fillna('Typ')
test_data.SaleType = test_data.SaleType.fillna(test_data.SaleType.value_counts().index[0])

test_data.Alley = test_data.Alley.fillna('None')
test_data.MasVnrType = test_data.MasVnrType.fillna('None')

test_data.BsmtQual = test_data.BsmtQual.fillna('None')
test_data.BsmtCond = test_data.BsmtCond.fillna('None')
test_data.BsmtExposure = test_data.BsmtExposure.fillna('None')
test_data.BsmtFinType1 = test_data.BsmtFinType1.fillna('None')
test_data.BsmtFinType2 = test_data.BsmtFinType2.fillna('None')

test_data.Electrical = test_data.Electrical.fillna(test_data.Electrical.value_counts().index[0])
test_data.FireplaceQu = test_data.FireplaceQu.fillna('None')
test_data.GarageType = test_data.GarageType.fillna('None')
test_data.GarageFinish = test_data.GarageFinish.fillna('None')
test_data.GarageQual = test_data.GarageQual.fillna('None')
test_data.GarageCond = test_data.GarageCond.fillna('None')
test_data.PoolQC =     test_data.PoolQC.fillna('None')
test_data.Fence =      test_data.Fence.fillna('None')
test_data.MiscFeature = test_data.MiscFeature.fillna('None')

In [200]:
train_data.Alley = train_data.Alley.fillna('None')
train_data.MasVnrType = train_data.MasVnrType.fillna('None')

train_data.BsmtQual = train_data.BsmtQual.fillna('None')
train_data.BsmtCond = train_data.BsmtCond.fillna('None')
train_data.BsmtExposure = train_data.BsmtExposure.fillna('None')
train_data.BsmtFinType1 = train_data.BsmtFinType1.fillna('None')
train_data.BsmtFinType2 = train_data.BsmtFinType2.fillna('None')

train_data.Electrical = train_data.Electrical.fillna(train_data.Electrical.value_counts().index[0])
train_data.FireplaceQu = train_data.FireplaceQu.fillna('None')
train_data.GarageType = train_data.GarageType.fillna('None')
train_data.GarageFinish = train_data.GarageFinish.fillna('None')
train_data.GarageQual = train_data.GarageQual.fillna('None')
train_data.GarageCond = train_data.GarageCond.fillna('None')
train_data.PoolQC = train_data.PoolQC.fillna('None')
train_data.Fence = train_data.Fence.fillna('None')
train_data.MiscFeature = train_data.MiscFeature.fillna('None')

In [201]:
train_data = train_data.drop('Utilities' , axis = 1)

In [202]:
#++++++++++++++++++++++
#++++++++++++++++++++++
test_data = test_data.drop('Utilities' , axis = 1)

In [203]:
#将数字变为str类型 为转换为categorical 类型做准备
train_data.MSSubClass = train_data.MSSubClass.apply(str)
train_data.OverallCond = train_data.OverallCond.astype(str)

train_data.YrSold = train_data.YrSold.astype(str)
train_data.MoSold = train_data.MoSold.astype(str)

In [204]:
#++++++++++++++++++++++++
#++++++++++++++++++++++++
test_data.MSSubClass = test_data.MSSubClass.apply(str)
test_data.OverallCond = test_data.OverallCond.astype(str)

test_data.YrSold = test_data.YrSold.astype(str)
test_data.MoSold = test_data.MoSold.astype(str)

In [205]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')

for i in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(train_data[i].values)) 
    train_data[i] = lbl.transform(list(train_data[i].values))



In [206]:
#+++++++++++++++++++++++
#+++++++++++++++++++++++
for i in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(test_data[i].values)) 
    test_data[i] = lbl.transform(list(test_data[i].values))

In [163]:
test_data.shape

(1459, 78)

In [207]:
#=================
#boxcox除偏
numerical_features = train_data.dtypes[train_data.dtypes != object].index #含有预测值 SalePrice

skewed_feats = train_data[numerical_features].apply(lambda x: stats.skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


Skew in numerical features: 



,Skew
MiscVal,24.434913
PoolArea,15.932532
LotArea,12.560986
3SsnPorch,10.286510
LowQualFinSF,8.995688
LandSlope,4.805032
KitchenAbvGr,4.480268
BsmtFinSF2,4.247550
ScreenPorch,4.114690
BsmtHalfBath,4.095895


In [208]:
from scipy import special

for i in skewness[np.abs(skewness)>0.75].index:
    train_data[i] = special.boxcox1p(train_data[i] , 0.15)


In [209]:
#++++++++++++++++++++
#++++++++++++++++++++
test_skewness = skewness.copy()
test_skewness = test_skewness.drop('SalePrice')

for i in test_skewness[np.abs(test_skewness)>0.75].index:
    test_data[i] = special.boxcox1p(test_data[i] , 0.15)

In [211]:
train_data = pd.get_dummies(train_data)

In [212]:
#+++++++++++++++++++
#+++++++++++++++++++
test_data = pd.get_dummies(test_data)

In [218]:
print(train_data.shape , test_data.shape)

(1458, 220) (1459, 220)


In [214]:
comp = list(set(train_data.columns) - set(test_data.columns))
comp.remove('SalePrice')

In [215]:
for i in comp:
    test_data[i] = 0

In [ ]:
#==============
#==============

#==============
#测试区域

In [103]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

forest = RandomForestRegressor(n_estimators=100)
linear = LinearRegression()
svr = SVR()
dec_tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()
ridge = Ridge(alpha=3)
lasso = Lasso()

classifier = {'random_forest':forest , 'LR':linear , 'SVM':svr , 'decison_tree':dec_tree , 'KNN':knn , 'ridge':ridge , 'lasso':lasso}
#kf = KFold(train_data.shape[0] , 10 , True)

In [104]:
def rmse(model , data , label):
    #cv=5 分为5folds
    #在这里会对模型进行训练 
    return np.sqrt(-cross_val_score(model, data, label, scoring="neg_mean_squared_error", cv = 10))

In [223]:
vali_index = list(train_data.columns)
vali_index.remove('SalePrice')

In [224]:
best_model = _
smallest_rmse = 1.0
for model_name , model in classifier.items():
    result = np.average(rmse(model , train_data[vali_index] , train_data.SalePrice))
    if result<smallest_rmse:
        smallest_rmse = result
        best_model = model_name
        
    print(model_name , result)

print('\nbest:',best_model , smallest_rmse)
#真大

random_forest 0.015517610079411725
LR 18593351.040031657
SVM 0.03478950954698343
decison_tree 0.022448461849106812
KNN 0.026694002298243653
ridge 0.013007238361774694
lasso 0.04498399289592943

best: ridge 0.013007238361774694


In [687]:
from sklearn.model_selection import GridSearchCV

def grid_search_scoring(model , data , label):
    return np.average(rmse(model , data , label))

In [688]:
param_grid={'alpha':[0.5 , 1.0 , 1.5 , 2.0 , 2.5 , 3.0]}
kfold = KFold(train_data.shape[0] , 10 , True)
grid = GridSearchCV(estimator=ridge , param_grid=param_grid , scoring=grid_search_scoring , cv=kfold)

In [689]:
grid.fit(train_data[vali_index] , train_data.SalePrice)

GridSearchCV(cv=sklearn.cross_validation.KFold(n=1458, n_folds=10, shuffle=True, random_state=None),
       error_score='raise',
       estimator=Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function grid_search_scoring at 0x00000239A7E94158>,
       verbose=0)

In [691]:
grid.best_score_

0.189017025638297

In [225]:
#================
#================
#投入使用
ridge.fit(train_data[vali_index] , train_data.SalePrice)

Ridge(alpha=3, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [232]:
#进行预测
#生成结果
result = ridge.predict(test_data[vali_index])
result = np.exp(result)
result_csv = pd.DataFrame({'Id':test_id , 'SalePrice':np.exp(result)})
result_csv.to_csv('submit2.csv' , index=False)

In [230]:
result = np.exp(result)

In [231]:
result

array([22.06242109, 22.70301388, 23.10653168, ..., 22.99333764,
       21.95771734, 23.59991223])